In [ ]:
from pathlib import Path
import json
import pandas as pd
from uau_api.settings import Settings
from IPython.display import display, HTML
from requests.compat import urlparse, unquote
from uau_api import UauAPI
from uau_api.utils import write_jsonl

In [ ]:
wd = Path(Settings().WORKDIR)
uau = UauAPI(Settings().API_URL, Settings().API_KEY)
uau.authenticate('leonardo', 'hybr01')
spes = pd.read_excel(wd / 'NotasUAU/SPEs_Cadastro.xlsx')
obras_ativas = uau.Obras.obter_obras_ativas()
obras_ativas = pd.DataFrame(obras_ativas)
# obras_ativas = obras_ativas[obras_ativas['Empresa_obr'].isin(spes['Código'])]

In [ ]:

# resposta[0]

In [ ]:
arquivo = [{'ConteudoArquivo': '...',
  'NomeArquivo': 'NF ENGETRES 2025.5_PAGE-0001.JPG',
  'CaminhoArquivo': 'NOTAS UAU\\GMW\\423 - ARAGARÇAS\\NF ENGETRES 2025.5_PAGE-0001.JPG',
  'Identificador': 'PROCESSO 12879-ADM',
  'Origem': 1,
  'NomeBucker': None}]

In [ ]:
planejamento = uau.Planejamento.consultar_estrutura_planejamento(
    empresa=401,
    obra='430C',
    produto='-3',
    contrato='-3',
    item='1.01',
    servico='HY2121',
    sequencia=0
)

In [ ]:
pd.json_normalize(planejamento)

In [ ]:
processos = uau.ProcessoPagamento.consultar_processos(
    empresa_obra_periodo={
        'EmpresaObra': [
            {'Empresa': 401, 'Obra': '430C'},
            {'Empresa': 401, 'Obra': 'AD401'},
        ],
        'PeriodoInicial': '2025-06-01T00:00:00.300Z',
        'PeriodoFinal': '2025-06-24T00:00:00.300Z',
    }
)

In [ ]:

pd.json_normalize(processos).iloc[0].to_dict()

In [ ]:
with open(wd / 'NotasUAU/processos_pagamento.jsonl') as fc:
    processos_pagamento = [json.loads(line) for line in fc.readlines()]
processos_pagamento = pd.json_normalize(processos_pagamento)

In [ ]:
custos = Path(
    '/mnt/c/Users/LeonardoAlves/OneDrive - Hy Brazil Energia S.A/Área de Trabalho/demonstrativo_custos_solares.txt'
)

In [ ]:
custos.parent

In [ ]:
despesas = pd.read_csv(custos, sep='\t', encoding='latin1')

In [ ]:
obras = [
    '411 - UFV ANAGÉ',
    '411A - UFV ANAGÉ A',
    '411B - UFV ANAGÉ B',
    '415 - UFV VITORIA DA CONQUISTA',
    '415A - UFV VITORIA DA CONQUISTA A',
    '415B - UFV VITORIA DA CONQUISTA B',
    '415C - UFV VITORIA DA CONQUISTA C',
    '415D - UFV VITORIA DA CONQUISTA D',
    '415E - UFV VITORIA DA CONQUISTA E',
    '416 - UFV QUIXERE',
    '416A - UFV QUIXERE A',
    '416B - UFV QUIXERE B',
    '417 - UFV MUCAMBO',
    '417A - UFV MUCAMBO A',
    '417B - UFV MUCAMBO B',
    '417C - UFV MUCAMBO C',
    '417D - UFV MUCAMBO D',
    '418 - UFV IBIAPINA 1',
    '418A - UFV IBIAPINA 1 A',
    '419 - UFV IBIAPINA 2-4',
    '419A - UFV IBIAPINA 2-4 A',
    '419B - UFV IBIAPINA2-4 B',
    '419C - UFV IBIAPINA 2-4 C',
    '420 - UFV BOM CONSELHO',
    '420A - UFV BOM CONSELHO A',
    '420B - UFV BOM CONSELHO B',
    '420C - UFV BOM CONSELHO C',
    '422 - UFV RUSSAS',
    '422A - UFV RUSSAS A',
    '422B - UFV RUSSAS B',
    '422C - UFV RUSSAS C',
    '422D - UFV RUSSAS D',
    '422E - UFV RUSSAS E',
    '423 - UFV ARAGARÇAS',
    '423A - UFV ARAGARCAS A',
    '423B - UFV ARAGARCAS B',
    '423C - UFV ARAGARCAS C',
    '423D - UFV ARAGARCAS D',
    '423E - UFV ARAGARCAS E',
    '424 - UFV VERTENTES 1',
    '424A - UFV VERTENTES A',
    '425 - UFV VERTENTES 2-3',
    '425B - UFV VERTENTES B',
    '425B - UFV VERTENTES C',
    '425C - UFV VERTENTES C',
    '428 - UFV MASSAPÊ',
    '428A - UFV MASSAPE A',
    '428B - UFV MASSAPE B',
    '428C - UFV MASSAPE C',
    '428D - UFV MASSAPE D',
    '428E - UFV MASSAPE E',
    '429 - UFV COREAU',
    '429A - UFV COREAU A',
    '429B - UFV COREAU B',
    '429C - UFV COREAU C',
    '429D - UFV COREAU D',
    '429E - UFV COREAU E',
    '430 - UFV RIO BRANCO',
    '430A - UFV RIO BRANCO A',
    '430B - UFV RIO BRANCO B',
    '430C - UFV RIO BRANCO C',
    '430D - UFV RIO BRANCO D',
    '430E - UFV RIO BRANCO E',
    '435 - UFV BELO JARDIM 1-2',
    '435A - UFV BELO JARDIM A',
    '435B - UFV BELO JARDIM B',
    '437 - UFV BELO JARDIM 3-4',
    '437C - UFV BELO JARDIM C',
    '437D - UFV BELO JARDIM D',
]

In [ ]:
codigo_obras = list(map(lambda x: x.split(' - ')[0], obras))

In [ ]:
# Anagé
# Aragarças
# Belo Jardim 1-2
# Bom Conselhob Im ...
# Coreau
# Coreaú
# Ibiapina 2-4
# Massapê
# Mucambo
# Quixeré
# Rio Branco
# Russas
# Vertentes 1
# Vertentes 2-3
# Vitória da Conquista

In [ ]:
obras_ativas = obras_ativas[obras_ativas['Empresa_obr'] >= 254]
obras_ativas = obras_ativas[obras_ativas['Empresa_obr'] <= 500]

In [ ]:
obras_ativas = obras_ativas[obras_ativas['Cod_obr'].isin(codigo_obras)]

In [ ]:
empresa_obra_periodo = {
        'EmpresaObra': [{'Empresa': 33, 'Obra': 'ADM'}],
        'PeriodoInicial': '2025-04-01T00:00:00.300Z',
        'PeriodoFinal': '2025-05-20T00:00:00.300Z',
    }
processos = uau.ProcessoPagamento.consultar_processos(empresa_obra_periodo=empresa_obra_periodo)

In [ ]:
for _, row in obras_ativas[obras_ativas['Descr_obr'].str.contains('UFV')].iterrows():
    empresa_obra_periodo = {
        'EmpresaObra': [{'Empresa': row['Empresa_obr'], 'Obra': row['Cod_obr']}],
        'PeriodoInicial': '2003-04-01T00:00:00.300Z',
        'PeriodoFinal': '2025-05-20T00:00:00.300Z',
    }
    response = uau.ProcessoPagamento.consultar_processos(
        empresa_obra_periodo=empresa_obra_periodo
    )
    if response:
        write_jsonl(response, custos.parent / 'processos_pagamento_solares.jsonl', mode='a')

In [ ]:
processos_pagamento[0]['Parcelas'][0].keys()

In [ ]:
with open(wd / 'NotasUAU/downloaded.jsonl') as fc:
    file_contents = [json.loads(line) for line in fc.readlines()]

In [ ]:
df = pd.json_normalize(file_contents)

In [ ]:
def get_file_id(string):
    query = urlparse(string).query
    result = unquote(query)  # 273-420A-2-2024-04-10-00289348000736-63023-0.jpg
    return result.split('&', 1)[0].rsplit('/')[-1].split('.')[0]

In [ ]:
html = df.iloc[:10][['Empresa', 'Obra', 'to_download.Processo', 'url']].to_html(
    formatters={'url': lambda x: f'<a href="{x}">{get_file_id(x)}</a>'}, escape=False
)
display(HTML(html))